# Flux Qubits - Test Chip

This code creates a GDS file containing eight flux qubits, four floating and four grounded, each one with a flux bias line and a readout/control resonator coupled to the same reflection feedline.
In addition the chip has lithography marks and a label.

It uses the package `QubitDrawing`, based on the package `gdspy` (documentaion here: https://gdspy.readthedocs.io/en/stable/index.html).
The workflow in general is as follows:
1. Set design parameters
2. Create a Top gdspy cell
3. Add elements to the Top cell
    1. Call the right function in `QubitDrawing`
    2. Add the returned cell to the Top cell
    3. Save the returned cell's reference to add to Negative (metallization)
4. Create label and lithography marks
5. Add cells' references, label and lithography marks to Negative (metallization)
6. Add Negative to Top cell
7. Save GDS file

In [7]:
# -*- coding: utf-8 -*-
"""
Created on Mar 2020

@author: QUANTIC, Barcelona
"""

import gdspy as gds
import numpy as np
from QubitDrawing import SuppFunctions, qbdraw

## Design Parameters

The first paragraph contains the design parameters for the qubit itself. They are decided by exploration of the Hamiltonian, but are not used directly in the code (except of the Josephson junction area) and meant for archiving purposes only.

The next paragrpahs are where the circuit parameters get in: lines width, capacitor length etc.
Notice especially the "Rotates" parameter. This is a list containing False's and True's. Its length determines the number of qubits in the chip and its values determine the qubit's postion, above or below the feedline. 

In [8]:
"""
This code is supposed to use the gdspy-based library QubitDrawingFunctions in order to build a gds file of a qubit-cicuit chip.
Layer 2 is kept for the circuit shapes and layer 0 is the chip without them (boolean operator), i.e. the evaporated layer.
Layer 5 is the eBeam layer, for qubit elements etc.
"""

'''Design Parameters'''
CHIP_NAME = 'ExampleChip'
chip_size = [5000,5000]  
C_zeta = 21                     #[fF], shunting capacitance over the small Josephson junction
MagneticFluxRange = [0.45,0.55] #[Phi_0]=[Magnetic flux quantum]
JJarea = 0.09**2                #[μm^2], Josephson junction default area
J_c = 9                         #[μA/μm^2], Josephson junction critical current
C_j = 90*JJarea                 #[fF], Josephson junction total capacitance
alpha = 0.365                   #Coefficient the fourth junction's area
JJRelations = [1,1,1,np.sqrt(alpha)]     #Ratios between Josephson junctions' lengths in the qubit's loop
JJparameters=     {'FingerWidth': np.sqrt(JJarea), #Dictionary with the Josephson junction parameters
                   'FingerLength':  1,          #
                   'TaperWidth': 0.45,             #
                   'BridgeWidth': np.sqrt(JJarea)}
FourJJloopLength = 10 #Length (and width, unless other specified) of the qubit's loop
# FourJJloopWidth = 10 #Uncomment and specify for non-sqare loops

#4JJ Qubits
CapcitorWidth = 100
#4JJ Grounded Qubits
CapacitorLength = 500

#General Circuit Settings
FeedlineLength = 3750
FeedlineWidth = 10 
SpaceWidth = 5      #Space between feedline and grounding plane
eBeamWidth = 2
eBeamLayer = 5
QubitSpacing = 4*SpaceWidth #Spacing between capacitor plates or qubit loop and the ground plane 

#Characteristic impedance and effective relative permittivity of feedline and resonators
SubstrateEpsilon_r = 11.68   #Intrinsic Silicon
SubstrateHeight = 500        #[μm]
Z_0, epsilon_e = SuppFunctions.coplanar_waveguide(SubstrateEpsilon_r,SubstrateHeight,FeedlineWidth,SpaceWidth)

#Resonators and qubits below and above the feedline, alternately.
ResonatorsResonances=[]
Elongations=[220.71794167942616,208.2762043523108,196.63715975597705,185.7255554469142,175.47526048991568,165.8279240597995,156.73186399711855,148.14114060458647]
Rotates = [False,True,False,True,False,True,False,True]
x_Origins = np.linspace(-1300,1400,len(Rotates))   #x origins of the resonators

#Bias line
BiaslineWidth = 2
BiaslineSpaceWidth = 1 #Space between bias line and grounding plane
BiaslineLength = 300
TerminalWidth = 10 
Tshape = False          #False for one-sided antenna (With half the terminal width)
BiasDistance = (FourJJloopLength/2+QubitSpacing) #Distance of the antenna from the qubit cenetr [>=(FourJJloopLength/2+QubitSpacing)]
BiasAsymmetry = -FourJJloopLength/2-1 #Bias line not aligned with qubit

'''---End of design parameters---'''

'---End of design parameters---'

## Top Cell

This cell is the only cell that will be saved in the final GDS file. The other cells will be represented as references in it, in hierarchical order.

In [9]:
# define Circuit cell and its negative
Top = gds.Cell('TOP', exclude_from_current=True)

## Feedline

Call the right function and get a reflection feedline cell. Add the cell to the Top cell and save its reference at a certain point, named "origin". This reference will be added later to the Negative cell, which will be where metal is evaporized.

In [10]:
'''Feedline'''
Feedline = qbdraw.DrawReflectionFeedline(FeedlineCellName= 'ReflectionFeedline', MainlineLength = FeedlineLength, LineWidth = FeedlineWidth, SpaceWidth=SpaceWidth)
FeedlinePos = 250
FeedlineReference = gds.CellReference(Feedline[0], origin=(0, FeedlinePos))
Top.add(FeedlineReference)

This is what the cell would look like:
![ReflectionLine](ReflectionLine.PNG)

## Resonators, Qubits and Bias Lines

- Call the right function and get a resonator cell. Add the cell to the Top cell and save its reference at a certain point, named "origin". Notice that the origin change according to whether the resonator is below the feedline (Rotation=Flase) or above it (Rotation=True). This reference will be added later to the Negative cell, which will be where metal is evaporized.
- Call the right function and get a qubit cell. Add the cell to the Top cell and save its reference at a certain point, named "origin". Notice that the origin change according to whether the resonator is below the feedline (Rotation=Flase) or above it (Rotation=True). This reference will be added later to the Negative cell, which will be where metal is evaporized. Notice also thath the function called is different since the qubits above the feedline are floating and the qubits below it are grounded.
- Call the right function and get a biasline cell. Add the cell to the Top cell and save its reference at a certain point, named "origin". Notice that the origin change according to whether the resonator is below the feedline (Rotation=Flase) or above it (Rotation=True). This reference will be added later to the Negative cell, which will be where metal is evaporized. Notice also thath the function called is different since the qubits above the feedline are floating and the qubits below it are grounded.

In [16]:
'''Resonators, Qubits and Qubits Backgrounds'''
QubitDistance = 1350 #Distance from feedline (adjusted manually).  
ResonatorsReferences = []
QubitsBackgroundsReferences = []
BiaslinesReferences = []
for i in range(len(Rotates)):
    Resonator = qbdraw.DrawResonator(ResonatorCellName= 'Resonator_'+str(i), LineWidth = FeedlineWidth, SpaceWidth = SpaceWidth, num_meanders = 5, elongation = Elongations[i])
    if Rotates[i]:
        ResonatorReference = gds.CellReference(Resonator[0], origin=(x_Origins[i], FeedlinePos+SpaceWidth/2), rotation = 180)
    else:
        ResonatorReference = gds.CellReference(Resonator[0], origin=(x_Origins[i], FeedlinePos-SpaceWidth/2))
    Top.add(ResonatorReference)
    ResonatorsReferences.append(ResonatorReference)
    
    if Rotates[i]: #Floating qubit, above feedline
        Qubit4JJ = qbdraw.DrawFourJJqubit('4JJqubit_'+str(i), Spacing=QubitSpacing, JJRelations =  JJRelations, JJparameters=JJparameters,
                                     RectangleWidth=CapcitorWidth, FourJJloopLength=FourJJloopLength, LineWidth = eBeamWidth/2)
        qubit_origin = Qubit4JJ[2] #floating qubit origin with respect to the center of the cell center
        Qubit4JJReference = gds.CellReference(Qubit4JJ[0], origin=(x_Origins[i]-qubit_origin[0], FeedlinePos+QubitDistance-qubit_origin[1]), rotation = 180)  
        QubitBackgroundReference = gds.CellReference(Qubit4JJ[1], origin=(x_Origins[i], FeedlinePos+QubitDistance), rotation = 180)
    else:       #Grounded qubit, below feedline
        Qubit4JJ = qbdraw.DrawFourJJgroundedQubit('4JJqubit_'+str(i), Spacing=QubitSpacing, JJRelations =  JJRelations, JJparameters=JJparameters,
                                     RectangleWidth=CapcitorWidth, RectangleLength=CapacitorLength, FourJJloopLength=FourJJloopLength, LineWidth = eBeamWidth/2)
        Qubit4JJReference = gds.CellReference(Qubit4JJ[0], origin=(x_Origins[i], FeedlinePos-QubitDistance-(CapacitorLength-CapcitorWidth)))   
        QubitBackgroundReference = gds.CellReference(Qubit4JJ[1], origin=(x_Origins[i], FeedlinePos-QubitDistance-(CapacitorLength-CapcitorWidth)))
    Top.add(Qubit4JJReference)
    QubitsBackgroundsReferences.append(QubitBackgroundReference)
    
    if Rotates[i]:
        Biasline = qbdraw.DrawBiasLine(BiaslineCellName = 'Biasline_'+str(i), BiaslineLength = BiaslineLength, LineWidth = BiaslineWidth,
                                    SpaceWidth = BiaslineSpaceWidth, TerminalWidth=TerminalWidth, Tshape=Tshape, Rotation=-90, Galvanic=True)
        BiaslineReference = gds.CellReference(Biasline[0], origin=(x_Origins[i]+45+BiasDistance, FeedlinePos+QubitDistance-BiasAsymmetry), rotation = -90)
    else:
        Biasline = qbdraw.DrawBiasLine(BiaslineCellName = 'Biasline_'+str(i), BiaslineLength = BiaslineLength, LineWidth = BiaslineWidth,
                                    SpaceWidth = BiaslineSpaceWidth, TerminalWidth=TerminalWidth, Tshape=Tshape, Galvanic=True)
        BiaslineReference = gds.CellReference(Biasline[0], origin=(x_Origins[i]-BiasAsymmetry/2, FeedlinePos-QubitDistance-(CapacitorLength-CapcitorWidth)-FourJJloopLength), rotation = 90)
    Top.add(BiaslineReference)
    BiaslinesReferences.append(BiaslineReference)
  

| With Resonators | With Qubits | With Bias Lines |
| --- | --- | --- |
| ![Resonators](ReflectionLine_Resonators.PNG) | ![Qubits](ReflectionLine_Resonators_Qubits.PNG) | ![Biaslines](ReflectionLine_Resonators_Qubits_Biaslines.PNG) |

## Label and Lithography Marks

In addition to the metal-evaporized circuit elements, whose references are to be added to the Negative cell, we add a label and lithography marks in the shape of crosses. 

In [19]:
'''Label'''
label_position = (-chip_size[0]/2+400, -chip_size[1]/2+300)
Label = gds.Text('FLUX QUBITS TEST\nQUANTIC @ Glasgow', size = 100, position = label_position)
rect = gds.Rectangle(tuple(Label.get_bounding_box()[0]-[50,50]),tuple(Label.get_bounding_box()[1]+[50,50])) #Added spacing between rectangle and text
LabelNegative = gds.boolean(rect, Label, 'not')
LabelNegative.rotate(np.pi/2)
LabelNegative.translate(dx=-chip_size[0]+800,dy=-150)

'''Lithography marks array'''
crmk, mkar = qbdraw.CreateMarks(dx=chip_size[0]-300, dy=chip_size[1]-300)
Top.add(mkar)

## Negative

This cell is what will be evetuallt metal-evaporized. It is called Negative since the cells whose references it contains are actually what will NOT be evaporized, i.e. the photolithography mask. It does that using a boolean "not" operation.
Then the Negative is also added to the Top cell.

In [20]:
'''Negative (to be evaporated)'''
wafer = gds.Rectangle((-chip_size[0]/2, -chip_size[1]/2), (chip_size[0]/2, chip_size[1]/2) , layer=0)
Negative = gds.Cell('negative', exclude_from_current=True)
Negative.add(gds.boolean(wafer, [FeedlineReference]+ResonatorsReferences+QubitsBackgroundsReferences+BiaslinesReferences+[LabelNegative], 'not')) 
# print(len(Negative.get_polygons()))
Top.add(gds.CellReference(Negative, origin=(0, 0)))

## Save Cell to GDS file

Once all the cells are added to the Top cell, including the Negative cell, this cell can be saved into a GDS file.

In [21]:
qbdraw.saveCell2GDS([Top], CHIP_NAME)

Finally the saved gds file we get is:
![FullChip](FullChip.PNG) 
### Fin